In [1]:
# This mounts your Google Drive to the Colab VM.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Enter the foldername for the data set (added shortcut to 230/231N folder)
FOLDERNAME = 'Shareddrives/CS 230 231N/public_datasets'
assert FOLDERNAME is not None, "[!] Enter the foldername."

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys
dataset_path = '/content/drive/My Drive/{}'.format(FOLDERNAME)
# sys.path.append(dataset_path)

In [5]:
from concurrent.futures import ThreadPoolExecutor
import os

import numpy as np
import pandas as pd
import sklearn
from tqdm.auto import tqdm

In [7]:
# dataset_root_dir = '/content/drive/Shareddrives/CS 230 231N/sustainbench-main/sustainbench-main/dataset_preprocessing/dhs_lsms'

In [8]:
# TODO: copy dhs_final_labels.csv to VM, change os path below accordingly
df = pd.read_csv(os.path.join(dataset_root_dir, 'output_labels/dhs_final_labels.csv'))
df['survey'] = df['DHSID_EA'].str[:10]
df['cc'] = df['DHSID_EA'].str[:2]

# TODO: run modified version of get_public_datasets.py, change data_dir below to match VM path
data_dir = '/content/drive/Shareddrives/CS 230 231N/dhs_datasets/'
df['path'] = data_dir + df['survey'] + '/' + df['DHSID_EA'] + '.npz'
# df['path'] = dataset_root_dir + '/dhs_npzs/' + df['survey'] + '/' + df['DHSID_EA'] + '.npz'

path_years = df[['DHSID_EA', 'path', 'year']].apply(tuple, axis=1)
df.set_index('DHSID_EA', verify_integrity=True, inplace=True, drop=False) #had to add drop=False to keep column from disappearing  -- R
print(df['path'].iloc[0])
df.info()
display(df.head())

/content/drive/Shareddrives/CS 230 231N/dhs_datasets/AL-2008-5#/AL-2008-5#-00000001.npz
<class 'pandas.core.frame.DataFrame'>
Index: 117644 entries, AL-2008-5#-00000001 to ZW-2015-7#-00000400
Data columns (total 24 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   DHSID_EA          117644 non-null  object 
 1   cname             117644 non-null  object 
 2   year              117644 non-null  int64  
 3   lat               117644 non-null  float64
 4   lon               117644 non-null  float64
 5   n_asset           86936 non-null   float64
 6   asset_index       86936 non-null   float64
 7   n_water           87938 non-null   float64
 8   water_index       87938 non-null   float64
 9   n_sanitation      89271 non-null   float64
 10  sanitation_index  89271 non-null   float64
 11  under5_mort       105582 non-null  float64
 12  n_under5_mort     105582 non-null  float64
 13  women_edu         117062 non-null  float64
 14  wo

,DHSID_EA,cname,year,lat,lon,n_asset,asset_index,n_water,water_index,n_sanitation,...,women_bmi,n_women_edu,n_women_bmi,cluster_id,adm1fips,adm1dhs,urban,survey,cc,path
DHSID_EA,,,,,,,,,,,,,,,,,,,,,
AL-2008-5#-00000001,AL-2008-5#-00000001,AL,2008,40.822652,19.838321,18.0,2.430596,18.0,3.444444,18.0,...,24.365000,18.0,18.0,1,NaN,9999,R,AL-2008-5#,AL,/content/drive/Shareddrives/CS 230 231N/dhs_da...
AL-2008-5#-00000002,AL-2008-5#-00000002,AL,2008,40.696846,20.007555,20.0,2.867678,20.0,4.700000,20.0,...,23.104000,20.0,20.0,2,NaN,9999,R,AL-2008-5#,AL,/content/drive/Shareddrives/CS 230 231N/dhs_da...
AL-2008-5#-00000003,AL-2008-5#-00000003,AL,2008,40.750037,19.974262,18.0,2.909049,18.0,4.500000,18.0,...,22.387778,18.0,18.0,3,NaN,9999,R,AL-2008-5#,AL,/content/drive/Shareddrives/CS 230 231N/dhs_da...
AL-2008-5#-00000004,AL-2008-5#-00000004,AL,2008,40.798931,19.863338,19.0,2.881122,19.0,4.947368,19.0,...,27.084500,21.0,20.0,4,NaN,9999,R,AL-2008-5#,AL,/content/drive/Shareddrives/CS 230 231N/dhs_da...
AL-2008-5#-00000005,AL-2008-5#-00000005,AL,2008,40.746123,19.843885,19.0,2.546830,19.0,4.684211,19.0,...,24.523125,16.0,16.0,5,NaN,9999,R,AL-2008-5#,AL,/content/drive/Shareddrives/CS 230 231N/dhs_da...


Assuming we are connected to a VM that has all the data and has already run a modified version of get_public_datasets.py that unzipped files from "data" folder and placed them in the "dhs_datasets" folder:

In [ ]:
# Function: paths_to_X
# build input matrix X by iterating through each path in paths, load image, insert in X
def paths_to_X(paths):  # -> (N, C, H, W) model input X
  '''
    Args
    - paths: array (N, 1)
      - path: str, path to npz file containing single entry 'x'
        representing a (C, H, W) image

    Returns: X, input matrix (N, C, H, W)
    '''
  N = len(paths)  # should be 117644
  C, H, W = 8, 255, 255
  X = np.zeros((N, C, H, W))
  for n in range(N):
    npz_path = paths[n]
    img = np.load(npz_path)['x']  # shape (C, H, W)
    X[n, :, :, :] = img
  return X

In [ ]:
# label_cols = ['asset_index', 'under5_mort', 'women_bmi', 'women_edu', 'water_index', 'sanitation_index']
# label_cols = ['n_under5_mort']
label = "n_under5_mort"

In [ ]:
SPLITS = {
    'train': [
        'AL', 'BD', 'CD', 'CM', 'GH', 'GU', 'HN', 'IA', 'ID', 'JO', 'KE', 'KM',
        'LB', 'LS', 'MA', 'MB', 'MD', 'MM', 'MW', 'MZ', 'NG', 'NI', 'PE', 'PH',
        'SN', 'TG', 'TJ', 'UG', 'ZM', 'ZW'],
    'val': [
        'BF', 'BJ', 'BO', 'CO', 'DR', 'GA', 'GN', 'GY', 'HT', 'NM', 'SL', 'TD',
        'TZ'],
    'test': [
        'AM', 'AO', 'BU', 'CI', 'EG', 'ET', 'KH', 'KY', 'ML', 'NP', 'PK', 'RW',
        'SZ']
}
SPLITS['trainval'] = SPLITS['train'] + SPLITS['val']

In [ ]:
def get_train_and_test(label, trainsplit='train', testsplit='test'):
    train_dhsids = df.index[df['cc'].isin(SPLITS[trainsplit]) & df[label].notna()]
    test_dhsids = df.index[df['cc'].isin(SPLITS[testsplit]) & df[label].notna()]

    train_X_paths = df.loc[train_dhsids, 'path'].values.reshape(-1, 1)
    train_X = paths_to_X(train_X_paths)
    train_Y = df.loc[train_dhsids, label].values
    test_X_paths = df.loc[test_dhsids, 'path'].values.reshape(-1, 1)
    test_X = paths_to_X(test_X_paths)
    test_Y = df.loc[test_dhsids, label].values

    # knn.fit(train_X, train_Y)
    # preds = knn.predict(test_X)
    return train_X, train_Y, test_X, test_Y

In [ ]:
train_X, train_Y, val_X, val_Y = get_train_and_test(label, 'train', 'val') 
trainval_X, trainval_Y, test_X, test_Y = get_train_and_test(label, 'trainval', 'test')
print("train_X: ", train_X.shape)
print("train_Y: ", train_Y.shape)
print("trainval_X: ", trainval_X.shape)
print("trainval_Y: ", trainval_Y.shape)
print("val_X: ", val_X.shape)
print("val_Y: ", val_Y.shape)
print("test_X: ", test_X.shape)
print("test_Y: ", test_Y.shape)

# IGNORE AFTER THIS - NOT DONE


In [ ]:
# From assignment2 PyTorch.ipynb

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

USE_GPU = True
dtype = torch.float32 # We will be using float throughout this tutorial.

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss.
print_every = 100
print('using device:', device)

In [ ]:
# From assignment2 PyTorch.ipynb

def check_accuracy_part34(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [ ]:
# From assignment2 PyTorch.ipynb

def train_part34(model, optimizer, epochs=1):
    """
    Train a model using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: Nothing, but prints model accuracies during training.
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                check_accuracy_part34(loader_val, model)
                print()


In [ ]:
solver = Solver(model, 
                data,
                update_rule=update_rule,
                optim_config=optim_config,
                #lr_decay=lr_decay,
                num_epochs=num_epochs, 
                batch_size=batch_size,
                print_every=print_every)
tr = solver.train()

In [ ]:
# Visualize train + val accuracy
plt.subplot(2, 1, 1)
plt.title('Training loss')
plt.plot(solver.loss_history, 'o')
plt.xlabel('Iteration')

plt.subplot(2, 1, 2)
plt.title('Accuracy')
plt.plot(solver.train_acc_history, '-o', label='train')
plt.plot(solver.val_acc_history, '-o', label='val')
plt.plot([0.5] * len(solver.val_acc_history), 'k--')
plt.xlabel('Epoch')
plt.legend(loc='lower right')
plt.gcf().set_size_inches(15, 12)
plt.show()

In [ ]:
best_model = None
best_val = 0

# hyperparams

for None:
      model = None #TODO: model
      solver = Solver(model, 
                data,
                update_rule=update_rule,
                optim_config=optim_config,
                #lr_decay=lr_decay,
                num_epochs=num_epochs, 
                batch_size=batch_size,
                print_every=print_every,
                verbose=False)
      tr = solver.train()
      
      # TODO: re-format to appropriate data format
      train_acc = solver.check_accuracy(data['X_train'], data['y_train'])
      val_acc = solver.check_accuracy(data['X_val'], data['y_val'])
      
      # TODO: once hyperparams picked
      print('results')
      if val_acc > best_val:
        best_model = model
        best_val = val_acc


In [ ]:
# TODO: determine how to give predicted regression
y_val_pred = best_model.loss(data['X_test'])
y_test_pred = best_model.loss(data['X_val'])


#TODO: evaluation metric

#TODO: print results